In [96]:
import os
import warnings
from pprint import pprint
import descarteslabs as dl
import matplotlib.pyplot as plt
from matplotlib import colors
from osgeo import gdal
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from descarteslabs.client.services import Catalog
%matplotlib inline

## Define Raster Datasets & Boundary

In [ ]:
# *************FIND COUNTRY BOUNDARY ***************
matches = dl.places.find('burundi')
aoi = matches[0]
pprint(aoi)

shape = dl.places.shape(aoi['slug'], geom='low')

In [ ]:
new_mexico = dl.places.shape('north-america_united-states_new-mexico')
pprint(new_mexico['geometry'])

In [ ]:
burundi_bounding_box = {
  'coordinates': [
    [
      [55210.738, -255587.129],
      [55210.738, -494687.129],
      [261010.738, -255587.129],
      [261010.738, -494687.129]
    ]
  ],
  'type': 'Polygon'
}

In [ ]:
tiles = dl.raster.dltiles_from_shape(
    resolution= 60.0, 
    tilesize=2048, 
    pad=16, 
    shape=shape)
pprint(tiles['features'][0])
pprint("Total number of tiles for Zimbabwe: " + str(len(tiles['features'])))

In [ ]:
# Search
import json
feature_collection = dl.metadata.search(products=['landsat:LC08:01:RT:TOAR'], 
                                        start_time='2017-05-01',
                                        end_time='2017-05-31', 
                                        cloud_fraction=.01,
                                        place=aoi['slug'])

print (len(feature_collection['features']))

In [ ]:
north_east = 'landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_172062_20170701_20170701_01_RT_v1'
south_east = 'landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_172063_20170701_20170701_01_RT_v1'
north_west = 'landsat:LC08:01:T1:TOAR:meta_LC08_L1TP_173062_20170606_20170616_01_T1_v1'
south_west = 'landsat:LC08:01:T1:TOAR:meta_LC08_L1TP_173063_20170606_20170616_01_T1_v1'
ids = [south_east,south_west, north_east,north_west]

## Visualize selected Images

In [ ]:
viz_arr, viz_meta = dl.raster.ndarray(
    ids,
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Byte',
    resolution=60,
    cutline=shape['geometry']
)

In [ ]:
clipped_arr, clipped_meta = dl.raster.ndarray(
    ids,
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Byte',
    resolution=351.263936238588940,
    srs = "EPSG:32636",
    bounds=(55210.738, -494687.129, 261010.738, -255587.129)
)

In [ ]:
plt.figure(figsize=[16,16])
plt.imshow(clipped_arr)

In [ ]:
plt.figure(figsize=[16,16])
plt.imshow(viz_arr)

## Prepare raster data for analysis

In [ ]:
arr, meta = dl.raster.ndarray(
    ids,
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Float32',
    resolution=60,
    cutline=shape['geometry']
)

In [ ]:
clipped_arr, clipped_meta = dl.raster.ndarray(
    ids,
    bands=['red', 'green', 'blue', 'alpha'],
    scales=[[0,4000], [0, 4000], [0, 4000], None],
    data_type='Float32',
    resolution=351.263936238588940,
    srs = "EPSG:32636",
    bounds=(55210.738, -494687.129, 261010.738, -255587.129)
)

In [ ]:
rows, cols, n_bands = clipped_arr.shape 

In [97]:
# print(clipped_meta.keys())
geo_transform = clipped_meta['geoTransform']
print geo_transform

proj = 'PROJCS["WGS 84 / UTM zone 36N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",33],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32636"]]'


[55210.738, 351.263936239, 0.0, -255587.129, 0.0, -351.263936239]


### Classification 
We need a library of functions to rasterize our training data.

In [ ]:
def create_mask_from_vector(vector_data_path, cols, rows, geo_transform,
                            projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    print(vector_data_path)
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    print(data_source)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds


def vectors_to_raster(file_paths, rows, cols, geo_transform, projection):
    """Rasterize the vectors in the given directory in a single image."""
    labeled_pixels = np.zeros((rows, cols))
    print
    for i, path in enumerate(file_paths):
        label = i+1
        ds = create_mask_from_vector(path, cols, rows, geo_transform,
                                     projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
    return labeled_pixels


def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

Define training datasets:

In [ ]:
output_fname = "data/urban_classification.tiff"
train_data_path = "data/training/"

In [ ]:
files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
print(classes)
shapefiles = [os.path.join(train_data_path, f)
              for f in files if f.endswith('.shp')]

labeled_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform,
                                   proj)
print(shapefiles, rows, cols, geo_transform, proj)
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
training_samples = clipped_arr[is_train]

In [ ]:
classifier = RandomForestClassifier(n_jobs=-1)
classifier.fit(training_samples, training_labels)

In [ ]:
n_samples = rows*cols
flat_pixels = clipped_arr.reshape((n_samples, n_bands))
result = classifier.predict(flat_pixels)
classification = result.reshape((rows, cols))

In [ ]:
# urban_class = classification.astype(int)
write_geotiff(output_fname, classification, geo_transform, proj)
# classification.shape

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=[16,16])
plt.imshow(urban_class)

### Upload Classification to Platform

In [88]:
band = classification
print("Band Type={}".format(type(band[0][0])))
      
min = np.ndarray.max(band)
max = np.ndarray.min(classification)
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))


Band Type=<type 'numpy.float64'>
Min=2.000, Max=1.000


In [93]:
Catalog().add_product('Burundi_Urban', 
                      title='Burundi_Urban', 
                      description='Test upload.'
                     )

{u'data': {u'attributes': {u'description': u'Test upload.',
   u'read': [],
   u'title': u'Burundi_Urban'},
  u'id': u'7294028cc01114d89a473cf055d29dc5cd5ffe88:Burundi_Urban',
  u'meta': {u'owner': {u'email': None,
    u'name': u'Karla King',
    u'uuid': u'google-oauth2|101598335967461239621'},
   u'owner_type': u'user'},
  u'type': u'product'}}

In [94]:
Catalog().add_band(product_id='7294028cc01114d89a473cf055d29dc5cd5ffe88:Burundi_Urban', name='urban', srcband=1, nbits=64,dtype='Float64',type='class',data_range=[1.000,2.000],colormap_name='magma')


{u'data': {u'attributes': {u'colormap_name': u'magma',
   u'data_range': [1.0, 2.0],
   u'default_range': None,
   u'dtype': u'Float64',
   u'jpx_layer': 0,
   u'name': u'urban',
   u'nbits': 64,
   u'nodata': None,
   u'read': [],
   u'res_factor': 1,
   u'srcband': 1,
   u'srcfile': 0,
   u'type': u'class'},
  u'id': u'7294028cc01114d89a473cf055d29dc5cd5ffe88:Burundi_Urban:urban',
  u'meta': {u'owner': {u'email': None,
    u'name': u'Karla King',
    u'uuid': u'google-oauth2|101598335967461239621'},
   u'owner_type': u'user'},
  u'relationships': {u'product': {u'data': {u'id': u'7294028cc01114d89a473cf055d29dc5cd5ffe88:Burundi_Urban',
     u'type': u'product'}}},
  u'type': u'band'}}

In [95]:
image_on_disc = '/Users/karlaking/descarteslabs-code/poverty-predictor/data/urban_classification.tiff'
acquired_date = '2018-04-05'

Catalog().upload_image(image_on_disc,
                       '7294028cc01114d89a473cf055d29dc5cd5ffe88:Burundi_Urban',  
                       acquired=acquired_date)